# [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html)

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

## Load Data

In [10]:
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## Build Model

References:

* [How to build a Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)
* [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

### Model Class

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    # See https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.forward
    # for when to call it and when not
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"{logits = }\n{logits.sum() = }\n{pred_probab = }\n{pred_probab.sum() = }\n{y_pred =}")

logits = tensor([[-0.0242, -0.0870,  0.0359, -0.0489,  0.0141,  0.0559, -0.0439, -0.1489,
          0.0065, -0.0759]], grad_fn=<AddmmBackward0>)
logits.sum() = tensor(-0.3163, grad_fn=<SumBackward0>)
pred_probab = tensor([[0.1006, 0.0944, 0.1068, 0.0981, 0.1045, 0.1090, 0.0986, 0.0888, 0.1037,
         0.0955]], grad_fn=<SoftmaxBackward0>)
pred_probab.sum() = tensor(1.0000, grad_fn=<SumBackward0>)
y_pred =tensor([5])


### Model Layers

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())
# print(input_image)

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
# print(flat_image)

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
# print(hidden1)

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[ 0.0082, -0.2714,  0.6688,  0.3836,  0.6673,  0.1920, -0.2683,  0.4494,
          0.3979, -0.1095, -0.2655, -0.3911,  0.0025, -0.4683,  0.0651,  0.0982,
         -0.1855,  0.2180,  0.5290, -0.5138],
        [-0.2567, -0.0304,  0.1218,  0.4594,  0.2213,  0.5576, -0.1141,  0.4734,
          0.5448,  0.0151, -0.3243, -0.1507,  0.1663, -0.6645, -0.1399, -0.1683,
          0.2068,  0.2574,  0.3632, -0.0761],
        [-0.0393,  0.2258,  0.5506, -0.0125,  0.4699,  0.6143, -0.2816,  0.3903,
          0.3901, -0.0430, -0.5656, -0.2342,  0.0852, -0.2530,  0.0263,  0.0348,
         -0.0237,  0.5558,  0.5514, -0.1092]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0082, 0.0000, 0.6688, 0.3836, 0.6673, 0.1920, 0.0000, 0.4494, 0.3979,
         0.0000, 0.0000, 0.0000, 0.0025, 0.0000, 0.0651, 0.0982, 0.0000, 0.2180,
         0.5290, 0.0000],
        [0.0000, 0.0000, 0.1218, 0.4594, 0.2213, 0.5576, 0.0000, 0.47

In [8]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(f"{logits = }\n{pred_probab = }")


logits = tensor([[-0.3281, -0.1616,  0.1189,  0.0329,  0.4127, -0.0072, -0.1119, -0.1828,
         -0.0802,  0.1417],
        [-0.1824, -0.1146,  0.0094, -0.0514,  0.3715,  0.0474, -0.1888, -0.0969,
         -0.0247,  0.2712],
        [-0.1028, -0.1427,  0.2439, -0.0830,  0.3015, -0.0452, -0.1444, -0.1042,
         -0.1493,  0.3082]], grad_fn=<AddmmBackward0>)
pred_probab = tensor([[0.0718, 0.0848, 0.1122, 0.1030, 0.1505, 0.0989, 0.0891, 0.0830, 0.0920,
         0.1148],
        [0.0816, 0.0874, 0.0989, 0.0931, 0.1421, 0.1027, 0.0811, 0.0889, 0.0956,
         0.1285],
        [0.0879, 0.0845, 0.1244, 0.0897, 0.1317, 0.0931, 0.0843, 0.0878, 0.0839,
         0.1326]], grad_fn=<SoftmaxBackward0>)


### Model Parameters

References

* [torch.nn.Module.named_parameters](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_parameters)
* [torch.nn.parameter.Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#torch.nn.parameter.Parameter)

In [9]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0316, -0.0039, -0.0192,  ...,  0.0022, -0.0191, -0.0288],
        [ 0.0079,  0.0178,  0.0354,  ..., -0.0217, -0.0209,  0.0235]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0049,  0.0313], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0389, -0.0089,  0.0238,  ...,  0.0237, -0.0240, -0.0267],
        [ 0.0081,  0.0365, -0.0093,  ..., -0.0395,  0.0305,  0.0202]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

## Train Model

References

* [How to train a model](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)
* [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [15]:
learning_rate = 1e-3

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward() # NOTE: Could/Should it be loss.sum().backward()?
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # NOTE: Here pred is not mapped to probablities before argmax. It's an optimazation.
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    

In [16]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.554545  [   64/60000]
loss: 0.660268  [ 6464/60000]
loss: 0.444369  [12864/60000]
loss: 0.689090  [19264/60000]
loss: 0.608994  [25664/60000]
loss: 0.600873  [32064/60000]
loss: 0.630677  [38464/60000]
loss: 0.677018  [44864/60000]
loss: 0.660667  [51264/60000]
loss: 0.611687  [57664/60000]
Test Error: 
 Accuracy: 78.9%, Avg loss: 0.609166 

Epoch 2
-------------------------------
loss: 0.541510  [   64/60000]
loss: 0.646332  [ 6464/60000]
loss: 0.434081  [12864/60000]
loss: 0.680104  [19264/60000]
loss: 0.601980  [25664/60000]
loss: 0.594128  [32064/60000]
loss: 0.618697  [38464/60000]
loss: 0.672921  [44864/60000]
loss: 0.654568  [51264/60000]
loss: 0.601844  [57664/60000]
Test Error: 
 Accuracy: 79.3%, Avg loss: 0.600203 

Epoch 3
-------------------------------
loss: 0.529446  [   64/60000]
loss: 0.633534  [ 6464/60000]
loss: 0.424572  [12864/60000]
loss: 0.671590  [19264/60000]
loss: 0.595317  [25664/60000]
loss: 0.587740  [32064/600